# Resolucion SUDOKU por DNN

Functions Definition

In [1]:
import numpy as np
import pandas as pd
from tensorflow.contrib.keras import utils

In [2]:
def load_data(nb_train=40000, nb_test=10000, full=False):
    """
    Function to load data in the keras way.
    
    Parameters
    ----------
    nb_train (int): Number of training examples
    nb_test (int): Number of testing examples
    full (bool): If True, whole csv will be loaded, nb_test will be ignored
    
    Returns
    -------
    Xtrain, ytrain (np.array, np.array),
        shapes (nb_train, 9, 9), (nb_train, 9, 9): Training samples
    Xtest, ytest (np.array, np.array), 
        shapes (nb_test, 9, 9), (nb_test, 9, 9): Testing samples
    """
    # if full is true, load the whole dataset
    if full:
        sudokus = pd.read_csv('../input/sudoku.csv').values
    else:
        sudokus = next(
            pd.read_csv('D:\Work\sudoku.csv', chunksize=(nb_train + nb_test))
        ).values
        
    
    quizzes, solutions = sudokus.T
    
          
    
    flatX = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in quizzes])
    flaty = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in solutions])
    
    return (flatX[:nb_train], flaty[:nb_train]), (flatX[nb_train:], flaty[nb_train:])

(Xtrain, ytrain), (Xtest, ytest) = load_data(1,1,False)  # We won't use _. We will work directly with ytrain

print (ytrain)
print (Xtrain)
print (ytrain.shape)
print (Xtrain.shape)



[[[8 6 4 3 7 1 2 5 9]
  [3 2 5 8 4 9 7 6 1]
  [9 7 1 2 6 5 8 4 3]
  [4 3 6 1 9 2 5 8 7]
  [1 9 8 6 5 7 4 3 2]
  [2 5 7 4 8 3 9 1 6]
  [6 8 9 7 3 4 1 2 5]
  [7 1 3 5 2 8 6 9 4]
  [5 4 2 9 1 6 3 7 8]]]
[[[0 0 4 3 0 0 2 0 9]
  [0 0 5 0 0 9 0 0 1]
  [0 7 0 0 6 0 0 4 3]
  [0 0 6 0 0 2 0 8 7]
  [1 9 0 0 0 7 4 0 0]
  [0 5 0 0 8 3 0 0 0]
  [6 0 0 0 0 0 1 0 5]
  [0 0 3 5 0 8 6 9 0]
  [0 4 2 9 1 0 3 0 0]]]
(1, 9, 9)
(1, 9, 9)


In [3]:
def delete_digits(X, n_delet=1):
    """
    This function is used to create sudoku quizzes from solutions
    
    Parameters
    ----------
    X (np.array), shape (?, 9, 9, 9|10): input solutions grids.
    n_delet (integer): max number of digit to suppress from original solutions
    
    Returns
    -------
    grids: np.array of grids to guess in one-hot way. Shape: (?, 9, 9, 10)
    """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, n_delet)] = 0  # generate blanks (replace = True)
        
    return np.reshape(utils.to_categorical(grids), (-1,9,9,10))

In [4]:
def diff(grids_true, grids_pred):
    """
    This function shows how well predicted quizzes fit to actual solutions.
    It will store sum of differences for each pair (solution, guess)
    
    Parameters
    ----------
    grids_true (np.array), shape (?, 9, 9): Real solutions to guess in the digit repesentation
    grids_pred (np.array), shape (?, 9, 9): Guesses
    
    Returns
    -------
    diff (np.array), shape (?,): Number of differences for each pair (solution, guess)
    """
    return (grids_true != grids_pred).sum((1, 2))

def batch_smart_solve(grids, solver):
    """
    NOTE : This function is ugly, feel free to optimize the code ...
    
    This function solves quizzes in the "smart" way. 
    It will fill blanks one after the other. Each time a digit is filled, 
    the new grid will be fed again to the solver to predict the next digit. 
    Again and again, until there is no more blank
    
    Parameters
    ----------
    grids (np.array), shape (?, 9, 9): Batch of quizzes to solve (smartly ;))
    solver (keras.model): The neural net solver
    
    Returns
    -------
    grids (np.array), shape (?, 9, 9): Smartly solved quizzes.
    """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        preds = np.array(solver.predict(np.reshape(utils.to_categorical(grids), (-1,9,9,10))))  # get predictions
        probs = preds.max(2).T  # get highest probability for each 81 digit to predict
        values = preds.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            if any(zero):  # don't try to fill already completed grid
                where = np.where(zero)[0]  # focus on blanks only
                confidence_position = where[prob[zero].argmax()]  # best score FOR A ZERO VALUE (confident blank)
                confidence_value = value[confidence_position]  # get corresponding value
                grid.flat[confidence_position] = confidence_value  # fill digit inplace
    return grids


Prepare data for the training phase.

In [16]:
input_shape = (9, 9, 10)
(_, ytrain), (Xtest, ytest) = load_data()  # We won't use _. We will work directly with ytrain

# one-hot-encoding --> shapes become :
# (?, 9, 9, 10) for Xs
# (?, 9, 9, 9) for ys

Xtrain = utils.to_categorical(ytrain).astype('float32')  # from ytrain cause we will creates quizzes from solusions
Xtest = utils.to_categorical(Xtest).astype('float32')
XtestTrue = utils.to_categorical(ytest).astype('float32')

ytest_true = np.copy(ytest)
ytrain = utils.to_categorical(ytrain-1).astype('float32') # (y - 1) because we 
ytest = utils.to_categorical(ytest-1).astype('float32')   # don't want to predict zeros




ytrain=np.reshape(ytrain, (-1,9,9,9))
Xtrain=np.reshape(Xtrain, (-1,9,9,10))
ytest=np.reshape(ytest, (-1,9,9,9))
Xtest=np.reshape(Xtest, (-1,9,9,10))
XtestTrue=np.reshape(XtestTrue, (-1,9,9,10))



# TensorFlow Keras

### Create the Model

In [6]:
from tensorflow.contrib.keras import models

In [7]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [8]:
from tensorflow.contrib.keras import layers

In [9]:
input_shape = (9,9,10)

In [10]:
dnn_keras_model.add(layers.Dense(units=64,input_shape=input_shape,activation='relu'))

W0902 19:45:15.112726  8628 deprecation.py:506] From C:\Anaconda\envs\tensorflow_env\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
dnn_keras_model.add(layers.Dropout(0.4))
dnn_keras_model.add(layers.Dense(units=64,activation='relu'))
dnn_keras_model.add(layers.Dropout(0.4))
dnn_keras_model.add(layers.Flatten())

In [12]:
grid = layers.Input(shape=input_shape)
features = dnn_keras_model(grid)
digit_placeholders = [ 
    layers.Dense(9, activation='softmax')(features)
    for i in range(81)
]

solver = models.Model (grid, digit_placeholders)

In [13]:
print(solver.summary())
#from tensorflow.contrib.keras import utils
#import graphviz 
#import pydotplus 
#utils.pydot = pydotplus
#utils.plot_model(solver, to_file='model_plot2.png', show_shapes=True, show_layer_names=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9, 9, 10)]   0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 5184)         4864        input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 9)            46665       sequential[1][0]                 
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 9)            46665       sequential[1][0]                 
______________________________________________________________________________________________

### Compile the Model

In [14]:
from tensorflow.contrib.keras import losses,optimizers,metrics, callbacks

In [15]:
solver.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [17]:
solver.fit(
    delete_digits(Xtrain, 0),  # we don't delete any digit for now
    [ytrain[:, i, j, :] for i in range(9) for j in range(9)],  # each digit of solution
    batch_size=128,
    epochs=1,  # 1 epoch should be enough for the task
    verbose=1,
)

40000/40000 [==============================] - 102s 3ms/sample - loss: 43.1880 - dense_2_loss: 0.5314 - dense_3_loss: 0.5356 - dense_4_loss: 0.5240 - dense_5_loss: 0.5321 - dense_6_loss: 0.5332 - dense_7_loss: 0.5314 - dense_8_loss: 0.5289 - dense_9_loss: 0.5285 - dense_10_loss: 0.5295 - dense_11_loss: 0.5321 - dense_12_loss: 0.5362 - dense_13_loss: 0.5353 - dense_14_loss: 0.5355 - dense_15_loss: 0.5288 - dense_16_loss: 0.5335 - dense_17_loss: 0.5323 - dense_18_loss: 0.5423 - dense_19_loss: 0.5360 - dense_20_loss: 0.5294 - dense_21_loss: 0.5353 - dense_22_loss: 0.5368 - dense_23_loss: 0.5331 - dense_24_loss: 0.5337 - dense_25_loss: 0.5308 - dense_26_loss: 0.5246 - dense_27_loss: 0.5296 - dense_28_loss: 0.5313 - dense_29_loss: 0.5311 - dense_30_loss: 0.5424 - dense_31_loss: 0.5382 - dense_32_loss: 0.5340 - dense_33_loss: 0.5298 - dense_34_loss: 0.5340 - dense_35_loss: 0.5275 - dense_36_loss: 0.5374 - dense_37_loss: 0.5265 - dense_38_loss: 0.5321 - dense_39_loss: 0.5259 - dense_40_loss: 

In [17]:
# evaluate the model
scores = solver.evaluate(
    delete_digits(XtestTrue, 1),  # we don't delete any digit for now
    [ytest[:, i, j, :] for i in range(9) for j in range(9)],
    verbose=0)

In [18]:
for n in range(81):
    print("%s: %.2f%%" % (solver.metrics_names[n], scores[n]*100))

loss: 17891.50%
dense_2_loss: 220.83%
dense_3_loss: 220.86%
dense_4_loss: 220.14%
dense_5_loss: 221.06%
dense_6_loss: 221.51%
dense_7_loss: 221.07%
dense_8_loss: 221.73%
dense_9_loss: 220.97%
dense_10_loss: 220.09%
dense_11_loss: 221.28%
dense_12_loss: 221.14%
dense_13_loss: 220.51%
dense_14_loss: 220.70%
dense_15_loss: 220.60%
dense_16_loss: 221.86%
dense_17_loss: 220.83%
dense_18_loss: 222.58%
dense_19_loss: 220.69%
dense_20_loss: 220.67%
dense_21_loss: 220.04%
dense_22_loss: 220.46%
dense_23_loss: 220.14%
dense_24_loss: 220.37%
dense_25_loss: 221.44%
dense_26_loss: 220.57%
dense_27_loss: 221.54%
dense_28_loss: 220.50%
dense_29_loss: 219.97%
dense_30_loss: 221.55%
dense_31_loss: 220.95%
dense_32_loss: 220.83%
dense_33_loss: 220.73%
dense_34_loss: 221.26%
dense_35_loss: 219.68%
dense_36_loss: 221.27%
dense_37_loss: 221.03%
dense_38_loss: 220.91%
dense_39_loss: 220.65%
dense_40_loss: 220.16%
dense_41_loss: 221.40%
dense_42_loss: 220.78%
dense_43_loss: 220.87%
dense_44_loss: 220.52%
den

In [20]:
solver.save("d:\model_solverv2.h5")
print("Saved model to disk")

Saved model to disk


In [22]:
solver = models.load_model('model_solver.h5')

In [72]:
xpredict = delete_digits(XtestTrue, 2)
predictions = solver.predict(xpredict)

In [73]:
print ('*'*50,'valores originales')
values = xpredict.argmax(3)
print(values)

preds = np.array(predictions)
#print ('shape prediction=', preds.shape)
values = preds.argmax(2).T + 1 
#print ('shape values original=', values.shape)
values = np.reshape(np.array(values), (-1,9,9))

print('*'*50, 'valores Predichos')
print(values)

print('*'*50, 'valores Realess')
print(ytest_true)



#zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions
#print(zeros)

************************************************** valores originales
[[[6 9 5 1 0 7 3 8 4]
  [1 3 8 4 0 9 6 7 2]
  [7 2 4 8 3 6 9 1 5]
  [8 5 1 2 6 4 7 3 9]
  [2 7 3 9 8 1 5 4 6]
  [9 4 6 5 7 3 8 2 1]
  [3 1 7 6 9 2 4 5 8]
  [4 8 9 7 1 5 2 6 3]
  [5 6 2 3 4 8 1 9 7]]

 [[0 9 7 2 5 8 3 1 6]
  [1 8 6 4 3 9 7 2 5]
  [2 5 3 7 1 6 4 9 8]
  [6 2 9 3 8 1 5 4 7]
  [3 7 5 9 6 4 1 8 2]
  [8 4 1 5 7 2 6 3 9]
  [9 6 2 1 4 5 8 7 3]
  [7 1 8 6 2 3 9 5 4]
  [5 0 4 8 9 7 2 6 1]]]
************************************************** valores Predichos
[[[6 9 5 1 5 7 3 8 4]
  [1 3 8 4 5 9 6 7 2]
  [7 2 4 8 3 6 9 1 5]
  [8 5 1 2 6 4 7 3 9]
  [2 7 3 9 8 1 5 4 6]
  [9 4 6 5 7 3 8 2 1]
  [3 1 7 6 9 2 4 5 8]
  [4 8 9 7 1 5 2 6 3]
  [5 6 2 3 4 8 1 9 7]]

 [[4 9 7 2 5 8 3 1 6]
  [1 8 6 4 3 9 7 2 5]
  [2 5 3 7 1 6 4 9 8]
  [6 2 9 3 8 1 5 4 7]
  [3 7 5 9 6 4 1 8 2]
  [8 4 1 5 7 2 6 3 9]
  [9 6 2 1 4 5 8 7 3]
  [7 1 8 6 2 3 9 5 4]
  [5 3 4 8 9 7 2 6 1]]]
************************************************** valores Re

In [19]:

early_stop = callbacks.EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],  # epochs for each round (17)
        [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]  # digit to pull off
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    solver.fit(
        delete_digits(Xtrain, nb_delete),  # delete digits from training sample
        [ytrain[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(Xtrain, nb_delete), # delete same amount of digit from validation sample
            [ytrain[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=128,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )


Pass n° 1 ...
Train on 40000 samples, validate on 40000 samples
40000/40000 [==============================] - 109s 3ms/sample - loss: 1.2475 - dense_2_loss: 0.0142 - dense_3_loss: 0.0167 - dense_4_loss: 0.0151 - dense_5_loss: 0.0155 - dense_6_loss: 0.0156 - dense_7_loss: 0.0147 - dense_8_loss: 0.0141 - dense_9_loss: 0.0169 - dense_10_loss: 0.0166 - dense_11_loss: 0.0149 - dense_12_loss: 0.0163 - dense_13_loss: 0.0157 - dense_14_loss: 0.0156 - dense_15_loss: 0.0165 - dense_16_loss: 0.0146 - dense_17_loss: 0.0148 - dense_18_loss: 0.0168 - dense_19_loss: 0.0155 - dense_20_loss: 0.0162 - dense_21_loss: 0.0151 - dense_22_loss: 0.0154 - dense_23_loss: 0.0145 - dense_24_loss: 0.0157 - dense_25_loss: 0.0153 - dense_26_loss: 0.0157 - dense_27_loss: 0.0159 - dense_28_loss: 0.0148 - dense_29_loss: 0.0153 - dense_30_loss: 0.0154 - dense_31_loss: 0.0153 - dense_32_loss: 0.0147 - dense_33_loss: 0.0156 - dense_34_loss: 0.0152 - dense_35_loss: 0.0156 - dense_36_loss: 0.0155 - dense_37_loss: 0.0150 - 

Pass n° 2 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/2
40000/40000 [==============================] - 105s 3ms/sample - loss: 1.4786 - dense_2_loss: 0.0172 - dense_3_loss: 0.0165 - dense_4_loss: 0.0180 - dense_5_loss: 0.0215 - dense_6_loss: 0.0179 - dense_7_loss: 0.0194 - dense_8_loss: 0.0179 - dense_9_loss: 0.0182 - dense_10_loss: 0.0190 - dense_11_loss: 0.0161 - dense_12_loss: 0.0193 - dense_13_loss: 0.0143 - dense_14_loss: 0.0180 - dense_15_loss: 0.0180 - dense_16_loss: 0.0187 - dense_17_loss: 0.0195 - dense_18_loss: 0.0175 - dense_19_loss: 0.0179 - dense_20_loss: 0.0208 - dense_21_loss: 0.0190 - dense_22_loss: 0.0192 - dense_23_loss: 0.0204 - dense_24_loss: 0.0198 - dense_25_loss: 0.0210 - dense_26_loss: 0.0188 - dense_27_loss: 0.0173 - dense_28_loss: 0.0195 - dense_29_loss: 0.0195 - dense_30_loss: 0.0176 - dense_31_loss: 0.0152 - dense_32_loss: 0.0174 - dense_33_loss: 0.0190 - dense_34_loss: 0.0183 - dense_35_loss: 0.0184 - dense_36_loss: 0.0193 - dense_37_loss:

Epoch 2/2
40000/40000 [==============================] - 114s 3ms/sample - loss: 1.0465 - dense_2_loss: 0.0121 - dense_3_loss: 0.0114 - dense_4_loss: 0.0115 - dense_5_loss: 0.0122 - dense_6_loss: 0.0108 - dense_7_loss: 0.0137 - dense_8_loss: 0.0137 - dense_9_loss: 0.0134 - dense_10_loss: 0.0120 - dense_11_loss: 0.0126 - dense_12_loss: 0.0125 - dense_13_loss: 0.0109 - dense_14_loss: 0.0130 - dense_15_loss: 0.0136 - dense_16_loss: 0.0137 - dense_17_loss: 0.0135 - dense_18_loss: 0.0136 - dense_19_loss: 0.0116 - dense_20_loss: 0.0162 - dense_21_loss: 0.0125 - dense_22_loss: 0.0130 - dense_23_loss: 0.0121 - dense_24_loss: 0.0104 - dense_25_loss: 0.0162 - dense_26_loss: 0.0133 - dense_27_loss: 0.0122 - dense_28_loss: 0.0134 - dense_29_loss: 0.0147 - dense_30_loss: 0.0129 - dense_31_loss: 0.0124 - dense_32_loss: 0.0142 - dense_33_loss: 0.0119 - dense_34_loss: 0.0128 - dense_35_loss: 0.0126 - dense_36_loss: 0.0124 - dense_37_loss: 0.0129 - dense_38_loss: 0.0113 - dense_39_loss: 0.0148 - dense_

Pass n° 3 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/3
40000/40000 [==============================] - 107s 3ms/sample - loss: 1.7165 - dense_2_loss: 0.0254 - dense_3_loss: 0.0207 - dense_4_loss: 0.0207 - dense_5_loss: 0.0185 - dense_6_loss: 0.0205 - dense_7_loss: 0.0201 - dense_8_loss: 0.0210 - dense_9_loss: 0.0226 - dense_10_loss: 0.0208 - dense_11_loss: 0.0213 - dense_12_loss: 0.0212 - dense_13_loss: 0.0192 - dense_14_loss: 0.0243 - dense_15_loss: 0.0207 - dense_16_loss: 0.0191 - dense_17_loss: 0.0218 - dense_18_loss: 0.0208 - dense_19_loss: 0.0211 - dense_20_loss: 0.0192 - dense_21_loss: 0.0234 - dense_22_loss: 0.0222 - dense_23_loss: 0.0232 - dense_24_loss: 0.0195 - dense_25_loss: 0.0204 - dense_26_loss: 0.0210 - dense_27_loss: 0.0226 - dense_28_loss: 0.0200 - dense_29_loss: 0.0199 - dense_30_loss: 0.0199 - dense_31_loss: 0.0231 - dense_32_loss: 0.0220 - dense_33_loss: 0.0232 - dense_34_loss: 0.0232 - dense_35_loss: 0.0206 - dense_36_loss: 0.0205 - dense_37_loss:

Epoch 2/3
40000/40000 [==============================] - 104s 3ms/sample - loss: 1.0981 - dense_2_loss: 0.0171 - dense_3_loss: 0.0123 - dense_4_loss: 0.0145 - dense_5_loss: 0.0132 - dense_6_loss: 0.0125 - dense_7_loss: 0.0134 - dense_8_loss: 0.0122 - dense_9_loss: 0.0133 - dense_10_loss: 0.0144 - dense_11_loss: 0.0145 - dense_12_loss: 0.0125 - dense_13_loss: 0.0146 - dense_14_loss: 0.0135 - dense_15_loss: 0.0117 - dense_16_loss: 0.0142 - dense_17_loss: 0.0139 - dense_18_loss: 0.0141 - dense_19_loss: 0.0144 - dense_20_loss: 0.0138 - dense_21_loss: 0.0150 - dense_22_loss: 0.0142 - dense_23_loss: 0.0132 - dense_24_loss: 0.0167 - dense_25_loss: 0.0119 - dense_26_loss: 0.0152 - dense_27_loss: 0.0132 - dense_28_loss: 0.0111 - dense_29_loss: 0.0115 - dense_30_loss: 0.0118 - dense_31_loss: 0.0151 - dense_32_loss: 0.0121 - dense_33_loss: 0.0149 - dense_34_loss: 0.0121 - dense_35_loss: 0.0161 - dense_36_loss: 0.0128 - dense_37_loss: 0.0140 - dense_38_loss: 0.0153 - dense_39_loss: 0.0141 - dense_

...

Epoch 3/10
40000/40000 [==============================] - 120s 3ms/sample - loss: 22.9293 - dense_2_loss: 0.2840 - dense_3_loss: 0.2859 - dense_4_loss: 0.2767 - dense_5_loss: 0.2802 - dense_6_loss: 0.2831 - dense_7_loss: 0.2811 - dense_8_loss: 0.2819 - dense_9_loss: 0.2808 - dense_10_loss: 0.2845 - dense_11_loss: 0.2818 - dense_12_loss: 0.2874 - dense_13_loss: 0.2865 - dense_14_loss: 0.2821 - dense_15_loss: 0.2840 - dense_16_loss: 0.2854 - dense_17_loss: 0.2810 - dense_18_loss: 0.2780 - dense_19_loss: 0.2860 - dense_20_loss: 0.2848 - dense_21_loss: 0.2859 - dense_22_loss: 0.2728 - dense_23_loss: 0.2777 - dense_24_loss: 0.2798 - dense_25_loss: 0.2857 - dense_26_loss: 0.2835 - dense_27_loss: 0.2859 - dense_28_loss: 0.2829 - dense_29_loss: 0.2842 - dense_30_loss: 0.2847 - dense_31_loss: 0.2786 - dense_32_loss: 0.2850 - dense_33_loss: 0.2815 - dense_34_loss: 0.2856 - dense_35_loss: 0.2815 - dense_36_loss: 0.2827 - dense_37_loss: 0.2825 - dense_38_loss: 0.2862 - dense_39_loss: 0.2852 - dens

Epoch 00003: early stopping
Pass n° 17 ...
Train on 40000 samples, validate on 40000 samples
Epoch 1/10
40000/40000 [==============================] - 119s 3ms/sample - loss: 31.8436 - dense_2_loss: 0.3895 - dense_3_loss: 0.3924 - dense_4_loss: 0.3874 - dense_5_loss: 0.3910 - dense_6_loss: 0.3907 - dense_7_loss: 0.3928 - dense_8_loss: 0.3929 - dense_9_loss: 0.3914 - dense_10_loss: 0.3919 - dense_11_loss: 0.3939 - dense_12_loss: 0.3953 - dense_13_loss: 0.3935 - dense_14_loss: 0.3923 - dense_15_loss: 0.3913 - dense_16_loss: 0.3936 - dense_17_loss: 0.3885 - dense_18_loss: 0.3964 - dense_19_loss: 0.3987 - dense_20_loss: 0.3952 - dense_21_loss: 0.3899 - dense_22_loss: 0.3902 - dense_23_loss: 0.3906 - dense_24_loss: 0.3929 - dense_25_loss: 0.3890 - dense_26_loss: 0.3927 - dense_27_loss: 0.3950 - dense_28_loss: 0.3871 - dense_29_loss: 0.3948 - dense_30_loss: 0.3912 - dense_31_loss: 0.3927 - dense_32_loss: 0.4007 - dense_33_loss: 0.3962 - dense_34_loss: 0.3925 - dense_35_loss: 0.3949 - dense_3

Epoch 2/10
40000/40000 [==============================] - 120s 3ms/sample - loss: 29.1327 - dense_2_loss: 0.3575 - dense_3_loss: 0.3607 - dense_4_loss: 0.3556 - dense_5_loss: 0.3583 - dense_6_loss: 0.3587 - dense_7_loss: 0.3610 - dense_8_loss: 0.3621 - dense_9_loss: 0.3581 - dense_10_loss: 0.3568 - dense_11_loss: 0.3602 - dense_12_loss: 0.3606 - dense_13_loss: 0.3611 - dense_14_loss: 0.3603 - dense_15_loss: 0.3554 - dense_16_loss: 0.3598 - dense_17_loss: 0.3552 - dense_18_loss: 0.3608 - dense_19_loss: 0.3651 - dense_20_loss: 0.3605 - dense_21_loss: 0.3543 - dense_22_loss: 0.3567 - dense_23_loss: 0.3560 - dense_24_loss: 0.3571 - dense_25_loss: 0.3572 - dense_26_loss: 0.3601 - dense_27_loss: 0.3607 - dense_28_loss: 0.3533 - dense_29_loss: 0.3593 - dense_30_loss: 0.3575 - dense_31_loss: 0.3616 - dense_32_loss: 0.3675 - dense_33_loss: 0.3600 - dense_34_loss: 0.3615 - dense_35_loss: 0.3614 - dense_36_loss: 0.3627 - dense_37_loss: 0.3650 - dense_38_loss: 0.3643 - dense_39_loss: 0.3593 - dens

Epoch 3/10
40000/40000 [==============================] - 120s 3ms/sample - loss: 27.9411 - dense_2_loss: 0.3452 - dense_3_loss: 0.3479 - dense_4_loss: 0.3400 - dense_5_loss: 0.3428 - dense_6_loss: 0.3432 - dense_7_loss: 0.3442 - dense_8_loss: 0.3490 - dense_9_loss: 0.3437 - dense_10_loss: 0.3438 - dense_11_loss: 0.3460 - dense_12_loss: 0.3455 - dense_13_loss: 0.3435 - dense_14_loss: 0.3472 - dense_15_loss: 0.3456 - dense_16_loss: 0.3452 - dense_17_loss: 0.3403 - dense_18_loss: 0.3468 - dense_19_loss: 0.3474 - dense_20_loss: 0.3465 - dense_21_loss: 0.3387 - dense_22_loss: 0.3443 - dense_23_loss: 0.3428 - dense_24_loss: 0.3410 - dense_25_loss: 0.3400 - dense_26_loss: 0.3440 - dense_27_loss: 0.3440 - dense_28_loss: 0.3387 - dense_29_loss: 0.3429 - dense_30_loss: 0.3444 - dense_31_loss: 0.3440 - dense_32_loss: 0.3539 - dense_33_loss: 0.3472 - dense_34_loss: 0.3457 - dense_35_loss: 0.3479 - dense_36_loss: 0.3467 - dense_37_loss: 0.3481 - dense_38_loss: 0.3496 - dense_39_loss: 0.3450 - dens

Epoch 00003: early stopping


In [ ]:
quizzes = Xtest.argmax(3)  # quizzes in the (?, 9, 9) shape. From the test set
true_grids = ytest.argmax(3) + 1  # true solutions dont forget to add 1 
smart_guesses = batch_smart_solve(quizzes, solver)  # make smart guesses !

deltas = diff(true_grids, smart_guesses)  # get number of errors on each quizz
accuracy = (deltas == 0).mean()  # portion of correct solved quizzes

In [ ]:
print(
"""
Grid solved:\t {}
Correct ones:\t {}
Accuracy:\t {}
""".format(
deltas.shape[0], (deltas==0).sum(), accuracy
)
)